## Note that, this code is experimental

You can import the generated .ply file to a 3DGS viewer to view it.

If you use this web-based 3DGS viewer: https://projects.markkellogg.org/threejs/demo_gaussian_splats_3d.php 
The recommanded initial camera settings are:
- Camera up: 	      0,1,0
- Camera position:    -1,100,500
- Camera look-at:     0,1,0


In [ ]:
## Enviroment Setup
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # Set the visible CUDA, here we use the second GPU
WORKING_DIR = '/home/peizhi/Documents/gaussian-dejavu/'
os.chdir(WORKING_DIR) # change the working directory to the project's absolute path
print("Current Working Directory: ", os.getcwd())
sys.path.append(WORKING_DIR)
sys.path.append('./models')
sys.path.append('./networks')
sys.path.append('./utils')
sys.path.append('./utils/flame_lib/')
sys.path.append('./utils/diff-gaussian-rasterization')
sys.path.append('./utils/gaussian_renderer')
sys.path.append('./utils/scene')
sys.path.append('./utils/arguments')
sys.path.append('./utils/simple-knn')

import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm

import numpy as np
import cv2
import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torchvision.transforms import Resize
# import math
# from torch.optim.lr_scheduler import StepLR

# # Dejavu
# from models.gaussian_dejavu_base import Framework
# from utils.loss import *
# from utils.scene.cameras import PerspectiveCamera, prepare_camera
# from utils.viewer_utils import OrbitCamera
# from utils.gaussian_renderer import render
# from utils.graphics_utils import create_diff_world_to_view_matrix, verts_clip_to_ndc
# from utils.loss_utils import *

# ## FLAME
# from utils.flame_lib import dict2obj
# from utils.flame_lib.FLAME import FLAME

import struct


# GUI
from ipywidgets import interactive, interact, Layout
from ipywidgets import FloatSlider
from ipywidgets import IntSlider
from ipywidgets import HBox, VBox, Label, Button


from dejavu import GaussianDejavu
from utils.personal_video_utils import *

def min_max_normalize(image):
    norm_image = (image - np.min(image)) / (np.max(image) - np.min(image))
    return norm_image

dejavu = GaussianDejavu(network_weights='./models/dejavu_network.pt', uv_map_size=120, num_expressions=20)

device = dejavu.device


## load head avatar

dejavu.load_head_avatar(save_path='./saved_avatars', avatar_name='peizhi-uv180')
#dejavu.load_head_avatar(save_path='./saved_avatars', avatar_name='imavatar-subject1-uv180')
#dejavu.load_head_avatar(save_path='./saved_avatars', avatar_name='imavatar-subject2-uv180')



Current Working Directory:  /home/peizhi/Documents/gaussian-dejavu
creating the FLAME Decoder


/home/peizhi/Documents/gaussian-dejavu/utils/flame_lib/FLAME.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('dynamic_lmk_faces_idx', torch.tensor(lmk_embeddings['dynamic_lmk_faces_idx'], dtype=torch.long))
/home/peizhi/Documents/gaussian-dejavu/utils/flame_lib/FLAME.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('dynamic_lmk_bary_coords', torch.tensor(lmk_embeddings['dynamic_lmk_bary_coords'], dtype=self.dtype))
/home/peizhi/miniconda3/envs/dejavu/lib/python3.10/site-packages/pytorch3d/io/obj_io.py:550: UserWarning: Mtl file does not exist: ./models/head_template2.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


Framework v3.1 initialized.
Number of Gaussians:  10694
model loaded from:  ./models/dejavu_network.pt
Gaussian DejaVu Framework Created.
Head avatar parameters loaded
Size of UV Gaussian map changed to 180x180
Number of Gaussians:  23873


In [ ]:


rendered, gaussians = dejavu.drive_head_avatar(exp = None, pose = None, eye_pose = None, cam_pose = None, return_gaussians = True)

xyz = gaussians[0].get_xyz.detach().cpu().numpy() * 900
rotation = gaussians[0].get_rotation.detach().cpu().numpy()[:, :4]
scale = gaussians[0].get_scaling.detach().cpu().numpy() * 180
sh = gaussians[0].get_features.detach().cpu().numpy()[:,0,:] # RGB colors instead of SH coefficients
opacity = gaussians[0].get_opacity.detach().cpu().numpy()

color = sh

print(xyz.shape, rotation.shape, scale.shape, sh.shape, opacity.shape)



(23873, 3) (23873, 4) (23873, 3) (23873, 3) (23873, 1)


In [ ]:


N = xyz.shape[0]  # Number of points

ply_file = './gaussians.ply'

"""
# Write PLY file in ASCII format
with open(ply_file, "w") as f:
    # PLY header
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write(f"element vertex {N}\n")
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    f.write("property float qw\n")
    f.write("property float qx\n")
    f.write("property float qy\n")
    f.write("property float qz\n")
    f.write("property float sx\n")
    f.write("property float sy\n")
    f.write("property float sz\n")
    f.write("property uchar red\n")
    f.write("property uchar green\n")
    f.write("property uchar blue\n")
    f.write("property uchar alpha\n")  # Opacity
    f.write("end_header\n")

    # Write each Gaussian to the file
    for i in range(N):
        f.write(f"{xyz[i, 0]} {xyz[i, 1]} {xyz[i, 2]} "  # XYZ
                f"{rotation[i, 0]} {rotation[i, 1]} {rotation[i, 2]} {rotation[i, 3]} "  # Quaternion
                f"{scale[i, 0]} {scale[i, 1]} {scale[i, 2]} "  # Scaling factors
                f"{color[i, 0]} {color[i, 1]} {color[i, 2]} {opacity[i]}\n")  # RGB + Opacity

print(f"PLY file saved: {ply_file}")
"""


# Define the PLY header (binary little-endian)
ply_header = f"""ply
format binary_little_endian 1.0
comment Created in Python
element vertex {N}
property float x
property float y
property float z
property float f_dc_0
property float f_dc_1
property float f_dc_2
property float opacity
property float scale_0
property float scale_1
property float scale_2
property float rot_0
property float rot_1
property float rot_2
property float rot_3
end_header
""".encode("utf-8")

# Open the file in binary mode
with open(ply_file, "wb") as f:
    # Write header
    f.write(ply_header)

    # Write binary data (little-endian format)
    for i in range(N):
        f.write(struct.pack("<3f", *xyz[i]))       # x, y, z
        f.write(struct.pack("<3f", *color[i]))      # f_dc_0, f_dc_1, f_dc_2
        f.write(struct.pack("<f", opacity[i][0])) # opacity
        f.write(struct.pack("<3f", *scale[i]))     # scale_0, scale_1, scale_2
        f.write(struct.pack("<4f", *rotation[i]))  # rot_0, rot_1, rot_2, rot_3

print(f"PLY file saved: {ply_file}")

PLY file saved: ./gaussians.ply
